In [67]:
#TODO:  after testing with sulfolobus, uncomment GTG in keep only valid orfs
import subprocess

def run_pipeline(file_prefix, input_path, output_path, SD): 
    
    # pre-process to get rid of bad orfs
    cds_file = input_path + file_prefix + "_cds_from_genomic.fna.gz"
    processed_cds_file = output_path + file_prefix + "_cds_processed.fa"
    try:
        ret = subprocess.check_output(["perl", "keep_only_valid_orfs.pl", cds_file, processed_cds_file], stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as exc:
        print("Problem executing keep_only_valid_orfs for ", cds_file, "; ", exc.returncode, exc.output)
    
    # get the codon pair scores of the cds
    codon_pair_scores_file = output_path + file_prefix + "_pairwise_syn_shuffle_frm123.csv"
    codon_pair_scores_helper_file = output_path + file_prefix + "_obs_exp_frm123.cts"
    try:
        ret = subprocess.check_output(["perl", "alisa_efficient_codon_pair_score_hexamers_paper.pl", "-s", \
                                       processed_cds_file, "-o", file_prefix, "-z", "2"], stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as exc:
        print("Problem executing alisa_efficient_codon_pair_score_hexamers for ", cds_file, "; ", exc.returncode, exc.output)
    
    # get the codon bias of the cds
    codon_bias_file = output_path + file_prefix + ".codon_bias"
    try:
        ps = subprocess.Popen("cat " + processed_cds_file + " | perl expected_codon_freq.pl >" + codon_bias_file, \
                              shell=True,stdout=subprocess.PIPE,stderr=subprocess.STDOUT)
        ret = ps.communicate()[0]
    except subprocess.CalledProcessError as exc:
        print("Problem executing expected_codon_freq.pl for ", cds_file, "; ", exc.returncode, exc.output)
                                      
    # collect the paper stats
    stats = ""
    
    # calculate the depletion score
    try:
        ret = subprocess.check_output(["perl", "get_atcg_freq.pl", "-i", processed_cds_file, "-s", SD])
        ret = float(ret.decode("utf-8"))
        try:
            stats += format(ret, '.3f') + ","
        except:
            print("Problem: get_atcg_freq.pl did not return a float for ", cds_file)
    except subprocess.CalledProcessError as exc:
        print("Problem executing get_atcg_freq.pl for ", cds_file, "; ", exc.returncode, exc.output)
                                       
    print(stats)
    # cleanup: remove created temporary files
    #TODO: rm processed_cds_file, codon_pair_scores_file, codon_pair_scores_helper_file codon_bias_file
    

input_file_path = "./"
output_file_path = "./"
file_prefix = "Sulfolobus_acidocaldarius"
SD = "GGTGA"
run_pipeline(file_prefix, input_file_path, output_file_path, SD)

-0.036,
